# Deploy Llama 3 70B with NVIDIA NIM on Amazon SageMaker

---

## What is NIM

NVIDIA NIM enables efficient deployment of large language models (LLMs) across various environments, including cloud, data centers, and workstations. It simplifies self-hosting LLMs by providing scalable, high-performance microservices optimized for NVIDIA GPUs. NIM's containerized approach allows for easy integration into existing workflows, with support for advanced language models and enterprise-grade security. Leveraging GPU acceleration, NIM offers fast inference capabilities and flexible deployment options, empowering developers to build powerful AI applications such as chatbots, content generators, and translation services.

### Features

NIM abstracts away model inference internals such as execution engine and runtime operations. They are also the most performant option available whether it be with TRT-LLM, vLLM or others. NIM offers the following high performance features:

1. Scalable Deployment that is performant and can easily and seamlessly scale from a few users to millions.
2. Advanced Language Model support with pre-generated optimized engines for a diverse range of cutting edge LLM architectures.
3. Flexible Integration to easily incorporate the microservice into existing workflows and applications. Developers are provided with an OpenAI API compatible programming model and custom NVIDIA extensions for additional functionality.
4. Enterprise-Grade Security emphasizes security by using safetensors, constantly monitoring and patching CVEs in our stack and conducting internal penetration tests.

### Architecture

NIMs are packaged as container images on a per model/model family basis. Each NIM is its own Docker container with a model, such as llama3. These containers include a runtime that runs on any NVIDIA GPU with sufficient GPU memory, but some model/GPU combinations are optimized. NIMs are distributed as NGC container images through the NVIDIA NGC Catalog. NIM automatically downloads the model from NGC, leveraging a local filesystem cache if available. Each NIM is built from a common base, so once a NIM has been downloaded, downloading additional NIMs is extremely fast.

In this example we show how to deploy `Llama3 70B` on a `p4d.24xlarge` instance with NIM on Amazon SageMaker.

## Model Card
---
### Llama 3 70B

- **Description:** Ideal for content creation, conversational AI, language understanding, research development, and enterprise applications. 
- **Max Tokens:** 2,048
- **Context Window:** 8,196
- **Languages:** English
- **Supported Use Cases:** Synthetic Text Generation and Accuracy, Text Classification and Nuance, Sentiment Analysis and Nuance Reasoning, Language Modeling, Dialogue Systems, and Code Generation.

## Prerequisites
---

<div class="alert alert-block alert-info">
<b>NOTE:</b>  To run NIM on SageMaker you will need to have your `NGC API KEY` to access NGC resources. Check out <a href="https://build.nvidia.com/meta/llama3-70b?signin=true"> this LINK</a> to learn how to get an NGC API KEY. 
</div>

##### 1. Setup and retrieve API key:

1. First you will need to sign into [NGC](9https://ngc.nvidia.com/signin) with your NVIDIA account and password.
2. Navigate to setup.
3. Select “Get API Key”.
4. Generate your API key.
5. Keep your API key secret and in a safe place. Do not share it or store it in a place where others can see or copy it

For more information on NIM, check out the [NIM LLM docs](https://docs.nvidia.com/nim/large-language-models/latest/introduction.html) .

##### 2. You must have `ecr:CreateRepository` and appropriate push permissions associated with your execution role

##### 3. NIM public ECR image is currently available only in `us-east-1` region

---

## Setup

Installs the dependencies and setup roles required to package the model and create SageMaker endpoint. 

In [1]:
import boto3 
import json
import os
import sagemaker
import time
from pathlib import Path
from sagemaker import get_execution_role

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")
region = sess.region_name
sts_client = sess.client('sts')
account_id = sts_client.get_caller_identity()['Account']

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Set Variables

In this example, since we are deploying `Llama3 70B` we define some configurations below for retrieving our ECR image for NIM along with some other requirements.

In [2]:
# llama-3-70b
public_nim_image = "public.ecr.aws/nvidia/nim:llama3-70b-instruct-1.0.0"
nim_model = "nim-llama3-70b-instruct"
sm_model_name = "nim-llama3-70b-instruct"
instance_type = "ml.p4d.24xlarge"
payload_model = "meta/llama3-70b-instruct"

### NIM Container

We first pull the NIM image from public ECR and then push it to private ECR repo within your account for deploying on SageMaker endpoint. 

Note, as mentioned previously:
  - NIM ECR image is currently available only in `us-east-1` region
  - You must have `ecr:CreateRepository` and appropriate push permissions associated with your execution role

In [3]:
import subprocess

# Get AWS account ID
result = subprocess.run(['aws', 'sts', 'get-caller-identity', '--query', 'Account', '--output', 'text'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode != 0:
    print(f"Error getting AWS account ID: {result.stderr}")
else:
    account = result.stdout.strip()
    print(f"AWS account ID: {account}")

bash_script = f"""
echo "Public NIM Image: {public_nim_image}"
docker pull {public_nim_image}


echo "Resolved account: {account}"
echo "Resolved region: {region}"

nim_image="{account}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Ensure the repository name adheres to AWS constraints
repository_name=$(echo "{nim_model}" | tr '[:upper:]' '[:lower:]' | tr -cd '[:alnum:]._/-')

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "$repository_name" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "$repository_name" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin "{account}.dkr.ecr.{region}.amazonaws.com"

docker tag {public_nim_image} $nim_image
docker push $nim_image
echo -n $nim_image
"""
nim_image=f"{account}.dkr.ecr.{region}.amazonaws.com/{nim_model}"
# Run the bash script and capture real-time output
process = subprocess.Popen(bash_script, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    output = process.stdout.readline()
    if output == b'' and process.poll() is not None:
        break
    if output:
        print(output.decode().strip())

stderr = process.stderr.read().decode()
if stderr:
    print("Errors:", stderr)


AWS account ID: 570598552974
Public NIM Image: public.ecr.aws/nvidia/nim:llama3-70b-instruct-1.0.0
llama3-70b-instruct-1.0.0: Pulling from nvidia/nim
cbe3537751ce: Already exists
d67fcc6ef577: Already exists
47ee674c5713: Already exists
63daa0e64b30: Already exists
d9d9aecefab5: Already exists
d71f46a15657: Pulling fs layer
054e2ffff644: Pulling fs layer
7d3cd81654d5: Pulling fs layer
dca613dca886: Pulling fs layer
0fdcdcda3b2e: Pulling fs layer
af7b4f7dc15a: Pulling fs layer
6d101782f66c: Pulling fs layer
e8427cb13897: Pulling fs layer
de05b029a5a2: Pulling fs layer
3d72a2698104: Pulling fs layer
aeff973c2191: Pulling fs layer
85d7d3ff0cca: Pulling fs layer
5996430251dd: Pulling fs layer
314dc83fdfc2: Pulling fs layer
5cef8f59ae9a: Pulling fs layer
927db4ce3e96: Pulling fs layer
cbe4a04f4491: Pulling fs layer
e8427cb13897: Waiting
ddb02ef514f8: Pulling fs layer
6d101782f66c: Waiting
de05b029a5a2: Waiting
a76bf8c63bc7: Pulling fs layer
4aadc1bd8c8a: Pulling fs layer
3d72a2698104: Waiti

We print the private ECR NIM image in your account that we will be using for SageMaker deployment. 
- Should be similar to  `"<ACCOUNT ID>.dkr.ecr.<REGION>.amazonaws.com/<NIM_MODEL>:latest"`

In [4]:
print(nim_image)

570598552974.dkr.ecr.us-west-2.amazonaws.com/nim-llama3-70b-instruct


---

## Create SageMaker Endpoint

**Before proceeding further, please set your NGC API Key.**

In [5]:
# Set your NGC API key here
NGC_API_KEY = "SET YOUR NGC API KEY"

In [6]:
assert NGC_API_KEY is not None, "NGC API KEY is not set. Please set the NGC_API_KEY variable. It's required for running NIM."

We define the sagemaker model from the NIM container making sure to pass in **NGC_API_KEY**

In [7]:
container = {
    "Image": nim_image,
    "Environment": {"NGC_API_KEY": NGC_API_KEY}
}
create_model_response = sm.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

Model Arn: arn:aws:sagemaker:us-west-2:570598552974:model/nim-llama3-70b-instruct


Next we create endpoint configuration, here we are deploying the Llama model on the specified instance type.

In [8]:
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            "ContainerStartupHealthCheckTimeoutInSeconds": 850
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint Config Arn: arn:aws:sagemaker:us-west-2:570598552974:endpoint-config/nim-llama3-70b-instruct


Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [9]:
endpoint_name = sm_model_name

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:570598552974:endpoint/nim-llama3-70b-instruct


In [10]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:570598552974:endpoint/nim-llama3-70b-instruct
Status: InService


## Test Inference and Streaming Inference with Endpoint

Once we have the endpoint's status as `InService` we can use a sample text to do a chat completion inference request using json as the payload format. For inference request format, currently NIM on SageMaker supports the OpenAI API chat completions inference protocol. For explanation of supported parameters please see [this link](https://platform.openai.com/docs/api-reference/chat). 

<div class="alert alert-block alert-info">
<b>NOTE:</b> The model's name in the inference request payload needs to be the name of the NIM model. 
</div>

In [11]:
messages = [
    {"role": "user", "content": "Hello! How are you?"},
    {"role": "assistant", "content": "Hi! I am quite well, how can I help you today?"},
    {"role": "user", "content": "Explain to me in detail how Optimum Neuron helps compile LLMs for AWS infrastructure"}
]
payload = {
  "model": payload_model,
  "messages": messages,
  "max_tokens": 1024
}


response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

{
  "id": "cmpl-304a3a7588be4edf83e96a3f3ea7720f",
  "object": "chat.completion",
  "created": 1722013379,
  "model": "meta/llama3-70b-instruct",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Optimum Neuron is an open-source library developed by Hugging Face that enables efficient compilation and deployment of Large Language Models (LLMs) on various infrastructure platforms, including AWS. Here's a detailed breakdown of how Optimum Neuron helps compile LLMs for AWS infrastructure:\n\n**Overview of the compilation process**\n\nWhen deploying an LLM on AWS infrastructure, the model needs to be optimized and compiled to run efficiently on the target hardware. This involves several steps:\n\n1. **Model loading**: The LLM is loaded from a model hub or a local file.\n2. **Model optimization**: The model is optimized for inference using techniques like pruning, quantization, and knowledge distillation to reduce its computational req

In [12]:
content = output["choices"][0]["message"]["content"]
print(content)

Optimum Neuron is an open-source library developed by Hugging Face that enables efficient compilation and deployment of Large Language Models (LLMs) on various infrastructure platforms, including AWS. Here's a detailed breakdown of how Optimum Neuron helps compile LLMs for AWS infrastructure:

**Overview of the compilation process**

When deploying an LLM on AWS infrastructure, the model needs to be optimized and compiled to run efficiently on the target hardware. This involves several steps:

1. **Model loading**: The LLM is loaded from a model hub or a local file.
2. **Model optimization**: The model is optimized for inference using techniques like pruning, quantization, and knowledge distillation to reduce its computational requirements.
3. **Model compilation**: The optimized model is compiled into an executable format that can run on the target AWS infrastructure.

**Optimum Neuron's role in compilation**

Optimum Neuron provides a uniform interface for compiling and deploying LLM

### Try streaming inference

NIM on SageMaker also supports streaming inference and you can enable that by setting **`"stream"` as `True`** in the payload and by using [`invoke_endpoint_with_response_stream`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint_with_response_stream.html) method.

In [13]:
messages = [
    {"role": "user", "content": "Hello! How are you?"},
    {"role": "assistant", "content": "Hi! I am quite well, how can I help you today?"},
    {"role": "user", "content": "Explain to me in detail what inference engines and llm serving frameworks are"}
]
payload = {
  "model": payload_model,
  "messages": messages,
  "max_tokens": 1024,
  "stream": True
}


response = client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
    Accept="application/jsonlines",
)

We have some postprocessing code for the streaming output.

In [14]:
event_stream = response['Body']
accumulated_data = ""
start_marker = 'data:'
end_marker = '"finish_reason":null}]}'

for event in event_stream:
    try:
        payload = event.get('PayloadPart', {}).get('Bytes', b'')
        if payload:
            data_str = payload.decode('utf-8')

            accumulated_data += data_str

            # Process accumulated data when a complete response is detected
            while start_marker in accumulated_data and end_marker in accumulated_data:
                start_idx = accumulated_data.find(start_marker)
                end_idx = accumulated_data.find(end_marker) + len(end_marker)
                full_response = accumulated_data[start_idx + len(start_marker):end_idx]
                accumulated_data = accumulated_data[end_idx:]

                try:
                    data = json.loads(full_response)
                    content = data.get('choices', [{}])[0].get('delta', {}).get('content', "")
                    if content:
                        print(content, end='', flush=True)
                except json.JSONDecodeError:
                    continue
    except Exception as e:
        print(f"\nError processing event: {e}", flush=True)
        continue

I'd be happy to explain inference engines and LLM serving frameworks in detail.

**Inference Engines**

An inference engine is a software component that enables machine learning models to make predictions or take actions based on input data. It is responsible for executing the model's computations and generating output. In other words, an inference engine is the "brain" that powers a machine learning model's decision-making process.

Inference engines are typically designed to work with specific types of machine learning models, such as neural networks, decision trees, or linear models. They can be used for various tasks, including:

1. ** Prediction**: Given input data, an inference engine can generate predictions or outputs based on the trained model.
2. **Scoring**: An inference engine can calculate the output score or probability of a particular input.
3. **Classification**: It can classify input data into predefined categories or labels.

An inference engine usually performs the f

---
### Delete endpoint and clean up artifacts

In [15]:
sm.delete_model(ModelName=sm_model_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '90b9f892-7fe9-446d-913e-31f52049dbd1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '90b9f892-7fe9-446d-913e-31f52049dbd1',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 26 Jul 2024 17:03:52 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

---
## Distributors
- Amazon Web Services
- Meta
